In [3]:
import glob
import numpy as np
from PIL import *
import PIL.Image

In [4]:
filelist = glob.glob('train/final_glu_aug/*.jpg')
glu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(glu.shape)

(357, 605, 700, 3)


In [5]:
filelist = glob.glob('train/final_non_glu_aug/*.jpg')
nonglu = np.array([np.array(Image.open(fname)) for fname in filelist])
print(nonglu.shape)

(357, 605, 700, 3)


In [6]:
X = np.concatenate((glu,nonglu),axis=0)
print(X.shape)

(714, 605, 700, 3)


In [7]:
gluL = np.array([1 for i in range(357)])
nongluL = np.array([0 for i in range(357)])
y = np.concatenate((gluL,nongluL),axis=0)
print(y.shape)

(714,)


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(478, 605, 700, 3) (236, 605, 700, 3) (478,) (236,)


In [9]:
from keras.models import Sequential
from keras.layers import Activation, Conv2D, MaxPooling2D, Flatten, Dropout,Dense
model = Sequential()

model.add(Conv2D(16, (3,3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3,3),padding='valid'))
model.add(Activation('relu'))

model.add(Conv2D(32, (3,3), strides =2))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), strides =2))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(32))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


Using TensorFlow backend.


In [10]:
model.output_shape

(None, 1)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 605, 700, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 605, 700, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 603, 698, 32)      4640      
_________________________________________________________________
activation_2 (Activation)    (None, 603, 698, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 301, 348, 32)      9248      
_________________________________________________________________
activation_3 (Activation)    (None, 301, 348, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 150, 174, 32)      0         
__________

In [17]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='mean_squared_error', optimizer=sgd, metrics = ['accuracy'])
# model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

In [18]:
# model.fit(X_train, y_train, batch_size=8, epochs=30, verbose=2, validation_data=(X_test, y_test))
model.fit(X_train, y_train, epochs=2, batch_size=1,validation_data=(X_test, y_test))

Train on 478 samples, validate on 236 samples
Epoch 1/2
478/478 [==============================] - 1707s 4s/step - loss: 0.4671 - acc: 0.5251 - val_loss: 0.5508 - val_acc: 0.4492
Epoch 2/2
478/478 [==============================] - 2166s 5s/step - loss: 0.4708 - acc: 0.5188 - val_loss: 0.5508 - val_acc: 0.4492


In [19]:
score = model.evaluate(X_test, y_test, batch_size=1)
print(score)

236/236 [==============================] - 272s 1s/step
[0.5508474576271186, 0.4491525423728814]


In [ ]:
model.predict(X_test, batch_size=1)
model.predict_classes(X_test,batch_size=1)